In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets
import xgboost as xgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/data-storm-10/credit_card_default_test.csv
/kaggle/input/data-storm-10/Credit_card_default - Business Problem - Assessment Criteria - Data Dictionary.xlsx
/kaggle/input/data-storm-10/credit_card_default_train.csv
/kaggle/input/data-storm-10/DATA STORM 1.0 - First Round Competition Guidlines.pdf


In [2]:
df = pd.read_csv("/kaggle/input/data-storm-10/credit_card_default_train.csv")
# test_dataset = pd.read_csv("/kaggle/input/data-storm-10/credit_card_default_test.csv")

In [3]:
df.head(5)

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
0,A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,...,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
1,A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,...,43530,80811,124590,151818,46200,43530,80811,942,33666,0
2,A4,100K,F,High School,Single,31-45,4,3,2,2,...,0,0,0,0,0,0,0,0,0,1
3,A5,200K,F,Graduate,Single,31-45,2,0,0,0,...,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
4,A6,1M,F,Graduate,Other,31-45,2,2,0,0,...,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [4]:
df.drop(['Client_ID'], 1, inplace=True)
df.fillna(0, inplace=True)

In [5]:
df.sample(5)

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
3512,200K,F,Other,Single,31-45,0,0,0,0,0,...,40330,41176,42026,4615,6930,1444,1495,1534,1499,0
23223,200K,M,High School,Single,46-65,1,2,0,0,0,...,64283,65306,66685,0,4043,4620,2772,2772,2613,0
12609,300K,M,Other,Other,Less than 30,1,2,0,0,0,...,225678,209298,197027,0,11550,6930,4881,6918,321,0
4736,100K,F,Other,Other,31-45,-1,-1,-1,0,-1,...,901,1802,901,3407,1802,0,72188,0,0,0
17772,100K,M,Other,Single,31-45,0,0,0,0,0,...,26429,26516,27071,2666,2740,3377,1155,924,1003,0


In [6]:
df["Balance_Limit_V1"].unique()

array(['1M', '100K', '200K', '400K', ' 500K', '300K', '1.5M', '2.5M'],
      dtype=object)

In [7]:
df["EDUCATION_STATUS"].unique()

array(['Graduate', 'High School', 'Other'], dtype=object)

In [8]:
df["MARITAL_STATUS"].unique()

array(['Other', 'Single'], dtype=object)

In [9]:
df["AGE"].unique()

array(['31-45', 'Less than 30', '46-65', 'More than 65'], dtype=object)

In [10]:
def map_age(age):
    if age == "Less than 30":
        return 0
    elif age == "31-45":
        return 1
    elif age == "46-65":
        return 2
    else:
        return 3

def map_balance(balance):
    if balance == "100K":
        return 1
    elif balance == "200K":
        return 2
    elif balance == "300K":
        return 3
    elif balance == "400K":
        return 4
    elif balance == "500K":
        return 5
    elif balance == "1M":
        return 10
    elif balance == "1.5M":
        return 15
    else:
        return 20


def map_education(education):
    if education == "Other":
        return 0
    elif education == "High School":
        return 1
    elif education == "Graduate":
        return 2

def map_gender(gender):
    if gender == "M":
        return 0
    return 1

def map_marital(marital):
    if marital == "Single":
        return 1
    return 0

In [11]:
df['Balance_Limit_V1'] = df.Balance_Limit_V1.apply(map_balance)
df['EDUCATION_STATUS'] = df.EDUCATION_STATUS.apply(map_education)
df['AGE'] = df.AGE.apply(map_age)
df['Gender'] = df.Gender.apply(map_gender)
df['MARITAL_STATUS'] = df.MARITAL_STATUS.apply(map_marital)

In [12]:
df.sample(10)

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
7550,2,1,0,0,0,-1,2,-1,0,0,...,2164,-7,2065,0,7256,5,0,2072,2093,0
13576,1,0,2,0,0,0,0,0,2,0,...,22405,22405,0,3012,5775,14,0,0,0,0
15211,15,0,0,1,1,0,0,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
13821,4,0,0,1,1,-2,-2,-2,-2,-2,...,6304,75191,19656,15521,63881,6304,75191,26489,16038,0
17200,10,0,2,0,0,-2,-2,-2,-2,-2,...,155003,11864,23830,16170,4507,155003,11878,23830,13213,0
3165,15,0,2,0,1,0,0,0,0,0,...,162301,162030,158249,9263,15523,11550,11550,9240,11550,0
6323,2,1,2,0,0,2,0,0,0,0,...,64897,67309,70744,5775,4620,2772,3465,4620,3465,1
14671,1,0,0,1,0,-1,-1,-1,-1,0,...,9868,9898,38212,26831,24807,9868,30,38547,2199,0
848,10,0,2,0,1,1,-2,-1,-1,-1,...,0,5449,0,0,7288,0,5449,0,0,0
13476,10,1,2,1,2,1,-2,-1,-1,-1,...,15893,1841,11314,0,60822,15893,1841,11314,8501,1


In [13]:
from sklearn import preprocessing
X = np.array(df.drop(['NEXT_MONTH_DEFAULT'], 1).astype(float))
# X = preprocessing.scale(X)
y = np.array(df['NEXT_MONTH_DEFAULT'])

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [15]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [16]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 20

In [17]:
model = xgb.train(param, D_train, steps)

In [18]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.7506955718969853
Recall = 0.6513576464335402
Accuracy = 0.8208333333333333


In [19]:
test_dataset = pd.read_csv("/kaggle/input/data-storm-10/credit_card_default_test.csv")
test_dataset['Balance_Limit_V1'] = test_dataset.Balance_Limit_V1.apply(map_balance)
test_dataset['EDUCATION_STATUS'] = test_dataset.EDUCATION_STATUS.apply(map_education)
test_dataset['AGE'] = test_dataset.AGE.apply(map_age)
test_dataset['Gender'] = test_dataset.Gender.apply(map_gender)
test_dataset['MARITAL_STATUS'] = test_dataset.MARITAL_STATUS.apply(map_marital)

In [20]:
test_dataset.sample()

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
5289,A26570,10,0,1,1,2,2,0,0,0,...,453938,278517,281044,284063,20790,14091,13860,10626,12705,6930


In [21]:
D_predict = xgb.DMatrix(np.array(test_dataset.drop(['Client_ID'], 1).astype(float)))
preds = model.predict(D_predict)

In [22]:
best_preds = np.asarray([np.argmax(line) for line in preds])

In [23]:
len(test_dataset['Client_ID']) == len(best_preds)

True

In [24]:
with open("output.csv", "w") as f:
    for i in range(len(preds)):
        f.write("{},{}\n".format(test_dataset['Client_ID'][i], best_preds[i]))